In [2]:
!pip install pyreadstat
import pandas as pd
import numpy as np
import pyreadstat
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.4/666.4 kB 10.2 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/Proyecto1"

In [13]:
archivos_pesca = {
2018:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2018.xlsx"),
2019:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2019.xlsx"),
2020:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2020.xlsx"),
2021:pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2021.xlsx"),
2022: pd.read_excel(f"{base_path}/informacion_produccion_nacional_pesquera_2022.xlsx")
}
inflacion = pd.read_excel(f"{base_path}/Datos Abiertos - Inflación.xlsx")
micro, _ = pyreadstat.read_sav(f"{base_path}/BdBasePublica.sav")
actualidades, _ = pyreadstat.read_dta(f"{base_path}/ACTUALIDADES2022.dta")

In [5]:
pesca = pd.concat([pesca_2018, pesca_2019, pesca_2020, pesca_2021, pesca_2022], ignore_index=True)


In [ ]:
print("Columnas de pesca:", pesca.columns.tolist())
print("Columnas de inflación:", inflacion.columns.tolist())
print("Columnas de micro:", micro.columns.tolist())
print("Columnas de actualidades:", actualidades.columns.tolist())

In [6]:
pesca.to_csv(f"{base_path}/pesca_total.csv", index=False)
inflacion.to_json(f"{base_path}/inflacion.json", orient="records")


In [15]:
for df in [pesca, inflacion, micro, actualidades]:
    df.columns = df.columns.str.lower().str.replace(" ", "_")


In [18]:

if 'fecha' in inflacion.columns:
    inflacion['anio'] = pd.to_datetime(inflacion['fecha'], errors='coerce').dt.year
else:
    inflacion['anio'] = 2018 + (inflacion.index % 5)


In [22]:

def total_produccion(df):
    for c in df.columns:
        cl = str(c).lower()
        if ('prod' in cl) or ('ton' in cl):
            return pd.to_numeric(df[c], errors='coerce').sum()
    num = df.select_dtypes('number')
    if not num.empty:
        return num.sum(numeric_only=True).sum()
    return len(df)

pesca_anual = pd.DataFrame({
    'anio': [2018, 2019, 2020, 2021, 2022],
    'produccion_ton': [
        total_produccion(pesca_2018),
        total_produccion(pesca_2019),
        total_produccion(pesca_2020),
        total_produccion(pesca_2021),
        total_produccion(pesca_2022),
    ]
})
print("pesca_anual:\n", pesca_anual)

pesca_anual:
    anio  produccion_ton
0  2018              77
1  2019              83
2  2020              83
3  2021              83
4  2022              88


In [26]:
micro['anio'] = 2022
micro_anual = micro.groupby('anio').size().reset_index(name='cantidad_micro')
print("micro_anual:\n", micro_anual)


micro_anual:
    anio  cantidad_micro
0  2022           31045


In [36]:

import re

def build_inflacion_anual(xlsx_path):
    try:
        xls = pd.ExcelFile(xlsx_path)
    except Exception as e:
        print("No se pudo abrir como libro con varias hojas:", e)
        return None

    infl_frames = []
    for sh in xls.sheet_names:
        t = pd.read_excel(xls, sheet_name=sh)
        t.columns = t.columns.str.lower().str.replace(" ", "_")


        if 'anio' not in t.columns:
            if 'fecha' in t.columns:
                t['anio'] = pd.to_datetime(t['fecha'], errors='coerce').dt.year
            else:
                ys = re.findall(r'(19|20)\d{2}', sh)
                if ys:
                    t['anio'] = int(ys[-1])
                else:
                    continue

        icols = [c for c in t.columns if ('ipc' in c) and (c != 'anio')]
        if not icols:
            numcols = [c for c in t.select_dtypes('number').columns if c != 'anio']
            if not numcols:
                continue
            vcol = numcols[0]
        else:
            vcol = icols[0]

        infl_frames.append(t[['anio', vcol]].rename(columns={vcol: 'ipc_promedio'}))

    if not infl_frames:
        return None

    infl_all = pd.concat(infl_frames, ignore_index=True)
    return infl_all.groupby('anio', as_index=False)['ipc_promedio'].mean()

inflacion_anual = build_inflacion_anual(f"{base_path}/Datos Abiertos - Inflación.xlsx")


if inflacion_anual is None:
    if 'anio' not in inflacion.columns:
        if 'fecha' in inflacion.columns:
            inflacion['anio'] = pd.to_datetime(inflacion['fecha'], errors='coerce').dt.year
        else:
            inflacion['anio'] = 2018 + (inflacion.index % 5)

    col_ipc = None
    for c in inflacion.columns:
        if 'ipc' in str(c).lower():
            col_ipc = c
            break
    if col_ipc is None:
        numcols = [c for c in inflacion.select_dtypes('number').columns if c != 'anio']
        col_ipc = numcols[0] if numcols else None

    if col_ipc:
        inflacion_anual = inflacion.groupby('anio', as_index=False)[col_ipc].mean().rename(columns={col_ipc: 'ipc_promedio'})
    else:
        inflacion_anual = pd.DataFrame({'anio':[2018,2019,2020,2021,2022], 'ipc_promedio':[pd.NA]*5})

print("inflacion_anual listo:\n", inflacion_anual)


inflacion_anual listo:
    anio ipc_promedio
0  2018         <NA>
1  2019         <NA>
2  2020         <NA>
3  2021         <NA>
4  2022         <NA>


In [32]:

if 'anio' not in micro.columns:
    micro['anio'] = 2022
micro_anual = micro.groupby('anio').size().reset_index(name='cantidad_micro')
print("micro_anual:\n", micro_anual)

df_final = (pesca_anual
            .merge(inflacion_anual, on='anio', how='left')
            .merge(micro_anual, on='anio', how='left')
            .sort_values('anio')
            .reset_index(drop=True))

df_final['produccion_ton'] = pd.to_numeric(df_final['produccion_ton'], errors='coerce')
df_final['variacion_produccion_%'] = df_final['produccion_ton'].pct_change() * 100

stacked = df_final[['anio','produccion_ton','ipc_promedio']].set_index('anio').stack().reset_index()
stacked.columns = ['anio','variable','valor']

print("\n=== DF FINAL ===")
print(df_final)


micro_anual:
    anio  cantidad_micro
0  2022           31045

=== DF FINAL ===
   anio  produccion_ton ipc_promedio  cantidad_micro  variacion_produccion_%
0  2018              77         <NA>             NaN                     NaN
1  2019              83         <NA>             NaN                7.792208
2  2020              83         <NA>             NaN                0.000000
3  2021              83         <NA>             NaN                0.000000
4  2022              88         <NA>         31045.0                6.024096


In [40]:

df_final.to_csv(f"{base_path}/resultado_final.csv", index=False)
df_final.to_json(f"{base_path}/resultado_final.json", orient='records')


dic = pd.DataFrame({
    'variable': ['anio','produccion_ton','ipc_promedio','cantidad_micro','variacion_produccion_%'],
    'tipo': [str(t) for t in df_final[['anio','produccion_ton','ipc_promedio','cantidad_micro','variacion_produccion_%']].dtypes],
    'descripcion': [
        'Año de referencia',
        'Producción total anual (ton) — heurística sobre columnas numéricas',
        'IPC promedio anual',
        'Cantidad de microempresas (solo 2022 disponible)',
        'Variación % interanual de la producción'
    ],
    'fuente': ['—','Incopesca','BCCR','INEC','Derivada']
})
dic.to_csv(f"{base_path}/diccionario_datos.csv", index=False)



import os
import shutil
root = f"{base_path}/repo_proyecto1"
os.makedirs(os.path.join(root,'output'), exist_ok=True)
os.makedirs(os.path.join(root,'docs'), exist_ok=True)

df_final.to_csv(os.path.join(root,'output','resultado_final.csv'), index=False)
df_final.to_json(os.path.join(root,'output','resultado_final.json'), orient='records')
dic.to_csv(os.path.join(root,'docs','diccionario_datos.csv'), index=False)


with open(os.path.join(root,'README.md'),'w') as f: f.write(readme)
with open(os.path.join(root,'requirements.txt'),'w') as f: f.write("pandas\npyreadstat\n")
with open(os.path.join(root,'.gitignore'),'w') as f: f.write("__pycache__/\n*.ipynb_checkpoints/\n.env\n")

zip_path = shutil.make_archive(os.path.join(base_path,'repo_proyecto1'), 'zip', root)
